In [ ]:
# Importing libraries
from cv2 import VideoCapture, VideoWriter, VideoWriter_fourcc, destroyAllWindows, inRange, erode, dilate, findContours, RETR_EXTERNAL, CHAIN_APPROX_SIMPLE, arcLength, minEnclosingCircle, circle, putText, FONT_HERSHEY_SIMPLEX, cvtColor, COLOR_BGR2HSV, COLOR_BGR2RGB
from PIL import Image
from IPython.display import display, clear_output
import numpy as np
import cv2
import time

# Defining color HSV ranges
color_ranges = {
    'Red':    [(0, 120, 70), (10, 255, 255), (170, 120, 70), (180, 255, 255)],
    'Green':  [(36, 100, 100), (86, 255, 255)],
    'Blue':   [(94, 80, 2), (126, 255, 255)],
    'Yellow': [(20, 100, 100), (30, 255, 255)],
    'White':  [(0, 0, 200), (180, 30, 255)],
    'Black':  [(0, 0, 0), (180, 255, 50)],
}

# Setup of webcam and output
cap = VideoCapture(0)
time.sleep(2.0)
frame_width, frame_height = 600, 400
fourcc = VideoWriter_fourcc(*'mp4v')
out = VideoWriter('color_ball_tracking.mp4', fourcc, 20.0, (frame_width, frame_height))

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (frame_width, frame_height))
        hsv = cvtColor(frame, COLOR_BGR2HSV)

        for color, bounds in color_ranges.items():
            if color == 'Red':
                mask1 = inRange(hsv, bounds[0], bounds[1])
                mask2 = inRange(hsv, bounds[2], bounds[3])
                mask = cv2.bitwise_or(mask1, mask2)
            else:
                mask = inRange(hsv, bounds[0], bounds[1])

            mask = erode(mask, None, iterations=2)
            mask = dilate(mask, None, iterations=2)

            cnts = findContours(mask.copy(), RETR_EXTERNAL, CHAIN_APPROX_SIMPLE)[0]

            for c in cnts:
                area = cv2.contourArea(c)
                if area > 500:
                    (x, y), radius = minEnclosingCircle(c)
                    if radius > 10:
                        perimeter = arcLength(c, True)
                        if perimeter == 0:
                            continue
                        circularity = 4 * np.pi * (area / (perimeter * perimeter))
                        if circularity > 0.6:
                            center = (int(x), int(y))
                            circle(frame, center, int(radius), (0, 255, 255), 2)
                            putText(frame, color, (center[0]-20, center[1]-20), FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        out.write(frame)
        frame_rgb = cvtColor(frame, COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        clear_output(wait=True)
        display(img)

except KeyboardInterrupt:
    print("Stopped manually.")

cap.release()
out.release()
destroyAllWindows()
print("Video saved as 'color_object_tracking.mp4'")